# Macroeconomic Variables
In questo codice viene eseguito il SARIMA su ua singola variabile macroeconomica per il calcolo dei suoi SHOCK.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# Si carica per esempio il dataset della Produzione Industriale.
Prod_Ind = pd.read_csv('Prod_Ind.csv', sep=';', index_col='Data', parse_dates=True)

Prod_Ind = Prod_Ind.pct_change()
Prod_Ind = Prod_Ind.dropna()

# Visualizzazione dataframe.
for col in Prod_Ind.columns:
    plt.plot(Prod_Ind.index, Prod_Ind[col], label=col)

plt.title("Variazione Percentuale di Prod Ind per le 6 nazioni")
plt.legend(loc="upper left", title="Legenda")
plt.show()

## Test Stazionarietà e differenziazioni
Viene testata la stazionarietà di tutte le serie con l'Augmented Dickey Fuller Test. Se almeno una serie risulta non stazionaria, si differenziano tutte le serie in modo da mantenere coerenza nel loro significato.

In [ ]:
# Test di stazionarietà e differenziazione automatica delle serie non stazionarie.
def ensure_stationarity(series_list):
    non_stationary_found = False  

    # Controllo della stazionarietà per ogni serie
    for series in series_list:
        p_value = test_stationarity(series)
        if p_value > 0.05: 
            non_stationary_found = True
            break

    if non_stationary_found:
        print("Almeno una serie non è stazionaria. Differenzio tutte le serie per mantenere il significato economico.")
        # Differenzia tutte le serie
        return {name: series.diff().dropna() for name, series in series_list.items()}
    else:
        print("Tutte le serie sono stazionarie.")
        return series_list

Prod_Ind_diff = pd.DataFrame()
for col in Prod_Ind.columns:
    print(f'Test di stazionarietà e differenziazione per {col}')
    Prod_Ind_diff[col] = ensure_stationarity(Prod_Ind[col])

Le serie risultanti sono le Variabili "GREZZE" su cui si calcolano gli shock. Si salvano.

In [ ]:
# Salvataggio variabili grezze
filename = 'prod_grezze.pkl'
Prod_Ind_diff.to_pickle(filename)

## Tuning dei Parametri SARIMA

Visualizzazione grafica delle componenti autoregressive e media mobile presenti nelle serie storiche con la Autocorrelation Function e la Partial Autocorrelation Function.

In [ ]:
def plot_acf_pacf(series, lags=40):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    plot_acf(series, lags=lags, ax=ax[0])
    plot_pacf(series, lags=lags, ax=ax[1])
    plt.show()

for col in Prod_Ind.columns:
    print(f'{col}')
    plot_acf_pacf(Prod_Ind_diff[col] , lags=20)
plt.show()

Calcolo dei parametri ideali trmite il Test di verifica delle informazioni di Akaike (AIC).

In [ ]:
def find_best_sarima(series, max_p=3, max_q=3, max_P=2, max_Q=2, max_s=4):
    best_aic = np.inf
    best_order = None
    best_seasonal_order = None
    best_mdl = None
    
    # d=0 in quanto le serie sono già stazionarie e in modo da mantenere lo stesso significato tra le serie.
    for p in range(max_p+1):
        for q in range(max_q+1):
            for P in range(max_P+1):
                for Q in range(max_Q+1):
                    for S in range(max_s+1):
                        
                        try:
                            mdl = SARIMAX(series, order=(p, 0, q), 
                                          seasonal_order=(P, 0, Q, S)).fit(disp=False)
                            # Verifica se questo modello ha il miglior AIC
                            if mdl.aic < best_aic:
                                best_aic = mdl.aic
                                best_order = (p, 0, q)
                                best_seasonal_order = (P, 0, Q, S)
                                best_mdl = mdl
                        except:
                            continue

    print(f'Miglior ordine SARIMA: {best_order} stagionale: {best_seasonal_order} con AIC: {best_aic}')
    return best_mdl

# Trova il miglior modello SARIMA per ogni serie differenziata.
# Si salvano i fitted values di ogni modello ottimale.
models = {}
predictions = {}
for col in Prod_Ind_diff.columns:
    print(f'Modello ARIMA ottimale per la serie {col}')
    model = find_best_sarima(Prod_Ind_diff[col])
    models[col] = model
    predictions[col] = model.fittedvalues


## Calcolo e salvataggio degli shock

In [ ]:
# Calcolo degli shock come differenza tra i valori osservati e le previsioni
shocks = pd.DataFrame()
for col in Prod_Ind_diff.columns:
    shocks[col] = Prod_Ind_diff[col] - predictions[col] 

shocks = shocks.dropna()

In [ ]:
# Salvataggio degli shock.
output_filename_pkl = 'prod_shocks.pkl'
shocks.to_pickle(output_filename_pkl)